# Data Generator

> Fill in a module description hered

In [ ]:
# | default_exp data_generator

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import re
from typing import List, Callable, Tuple

import torch
import torch.nn.functional as F

from torchtyping import TensorType
from langchain import PromptTemplate

from toolformer.api import BaseAPI
from toolformer.api import CalculatorAPI

/Users/education/DATA/projects/ai/toolformer/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export
class DataGenerator:
    def __init__(self, config: dict, model: Callable, tokenizer: Callable, apis: List[BaseAPI],):
        start_character = config["data_generator"]["api_start_character"]
        end_character = config["data_generator"]["api_end_character"]
        output_character = config["data_generator"]["api_output_character"]
        
        # add a space, because when the model generate a token, it's also include a "space"
        self.api_start_token = tokenizer(f' {start_character}', return_tensors="pt")["input_ids"][0]
        self.api_end_token = tokenizer(end_character, return_tensors="pt")["input_ids"][0]
        self.api_output_token = tokenizer(f'{output_character}', return_tensors="pt")["input_ids"][0]
        
        self.top_k = config["data_generator"]["top_k"]
        self.sampling_threshold = config["data_generator"]["sampling_threshold"]
        self.filtering_threshold = config["data_generator"]["filtering_threshold"]
        
        self.apis = apis
        self.model = model
        self.tokenizer = tokenizer
        # TODO: handle for cases that the sentence contains ".\n\n"
        self.eos_token_id = tokenizer(".\n\n")["input_ids"][0]
    
    def _sample_api_position(
        self,
        prompt_ids: TensorType["batch_size", "seq_len"], # the ids of the prompt
    ) -> Tuple[
        TensorType["batch_size", "n_positions"], # The positions of api call
        TensorType["batch_size", "seq_len"] # The generated text
    ]:
        # TODO: add support batch
        generated_ids = prompt_ids
        api_positions = torch.tensor([])
        
        with torch.no_grad():    
            while True:
                logits = self.model(
                    input_ids=generated_ids.unsqueeze(0),
                ).logits

                last_logit = logits[0, -1, :]
                probs = torch.softmax(last_logit, dim=-1)
                
                # find the top k tokens for api call
                # TODO: add filter by larger than sampling_threshold
                top_k_tokens = torch.topk(probs, k=5, dim=-1)
                
                if self.api_start_token in top_k_tokens.indices:
                    api_position = torch.tensor([len(generated_ids)]) # the current idx
                    api_positions = torch.cat((api_positions, api_position), dim=0)
                
                # sampling a token
                # next_token = torch.multinomial(probs, num_samples=1)
                next_token = torch.argmax(probs, dim=-1)
                next_token = next_token.unsqueeze(0)
                generated_ids = torch.cat([generated_ids, next_token], dim=0)
                
                if next_token == self.eos_token_id: break
        
        return api_positions.long(), generated_ids.long()

    def _obtain_api_call(
        self,
        positions: TensorType["batch_size", "n_positions"],
        generated_ids: TensorType["batch_size", "seq_len"]
    ) -> TensorType["batch_size", "n_positions", "seq_len"]:
        MAX_PAD = 1000
        PAD_TOKEN = self.tokenizer.pad_token_id
        candidates = torch.tensor([])

        for position in positions:
            text_ids = torch.cat([generated_ids[:position], self.api_start_token])
            output = self.model.generate(
                input_ids=text_ids.unsqueeze(0),
                eos_token_id=self.eos_token_id,
                max_new_tokens=100,
            )
            
            N_PAD = MAX_PAD - output.shape[-1]
            candidates = torch.cat([
                candidates,
                F.pad(output[0], pad=(0, N_PAD), value=PAD_TOKEN).unsqueeze(0).long()
            ], dim=0)
        return candidates.long()

    def extract_api_request_content(self, text: str, api_name: str) -> str:
        start_tag = f"{api_name}("
        end_tag = ")"
        start_idx = text.find(start_tag)
        if start_idx == -1:
            return None
        start_idx += len(start_tag)
        end_idx = text.find(end_tag, start_idx)
        if end_idx == -1:
            return None
        return text[start_idx:end_idx]
    
    def extract_api_syntax(self, sentence: str, api_name: str) -> str:
        pattern = r"\[{}\(.*?\)\]".format(api_name)
        matches = re.findall(pattern, sentence)
        return matches
    
    def _sampling_api(
        self,
        positions: TensorType["batch_size", "n_positions"],
        generated_ids: TensorType["batch_size", "seq_len"],
        prompt: PromptTemplate
    ):
        pass
    
    def _filter_api(
        self,
        prompt_ids: TensorType["batch_size", "seq_len"],
        candidates: List[int]
    ):
        calculator_api = CalculatorAPI()
        conditioning_prompts = []

        PROMPT_LENGTH = len(prompt_ids)
        SPACE_TOKEN = self.tokenizer(" .", return_tensors="pt")["input_ids"][0]
        API_NAME = "Calculator"

        for candidate in candidates:
            # the ids of the prediction
            pred_ids = candidate[PROMPT_LENGTH:]
            pred = self.tokenizer.decode(pred_ids, skip_special_tokens=True)
            
            api_request_content = self.extract_api_request_content(pred, api_name=API_NAME)
            api_response = calculator_api(api_request_content)
            api_response_with_arrow = torch.cat([self.api_output_token, self.tokenizer(api_response, return_tensors="pt")["input_ids"][0]], dim=0)
            
            api_syntax = self.extract_api_syntax(pred, api_name=API_NAME)
            api_syntax_ids = self.tokenizer(api_syntax, return_tensors="pt")["input_ids"][0]
            api_syntax_with_response_ids = torch.cat([api_syntax_ids[:-1], api_response_with_arrow, api_syntax_ids[-1:]])
            
            api_start_idx = torch.where(pred_ids == self.api_start_token)[0]
            pred_exclude_api_ids = pred_ids[:api_start_idx]
            next_token_ids = pred_ids[api_start_idx + 1]
            
            conditioning_prompts.append(torch.cat([api_syntax_with_response_ids, SPACE_TOKEN, pred_exclude_api_ids], dim=0))
                    
        return conditioning_prompts
    
    def generate(
        self,
        prompt_tempalte: PromptTemplate,
        text: str,
    ) -> List[str]:
        # TODO: add support batch
        prompt = prompt_tempalte.format(input=text)
        prompt_ids = self.tokenizer(prompt, return_tensors="pt")["input_ids"][0]  
        
        # sampling
        api_start_positions, generated_ids = self._sample_api_position(prompt_ids)
        
        # obtaining API calls
        candidates = self._obtain_api_call(api_start_positions, generated_ids)
    
        # filtering
        conditioning_prompts = self._filter_api(prompt_ids, candidates)
        
        return conditioning_prompts